In [2]:
using JuMP, HiGHS


In [17]:
# whiskey types
blended = [:Scottish, :Johnny]
raw = [:Standard, :Choice, :Prime]

#Sales price of 1 Liter of blended whiskeys
sellAt = Dict(zip(blended, [6.80, 5.70]))

#Price of 1 Liter of each raw whiskeys
purchaseAt = Dict(zip(raw, [4, 5, 7]))

#Amount available of each raw whiskey types
amount = Dict(zip(raw, [1200, 2500, 2000]))

#Demand of each blended whiskey type
demand = Dict(zip(blended, [1000, 600]))

Dict{Symbol, Int64} with 2 entries:
  :Johnny   => 600
  :Scottish => 1000

In [18]:

question2 = Model(HiGHS. Optimizer)

# Amount of Liter raw whiskey type used in each blended whiskey
@variable(question2, x[raw, blended] >= 0)

#Amount of money put into advertising for each type of blended whiskey
@variable(question2, y[blended] >= 0)

#Maximum amount of raw whiskey types available

@constraint(question2, amount[i in raw], sum(x[i,j] for j in blended) <= amount[i])

#Meet demand obligations with advertising increasing

@constraint(question2, demand[j in blended], sum(x[i,j] for i in raw) <= demand[j] + 1.25*y[j])
#Scottish Club must have at least 60% prime raw whiskey

@constraint(question2, SPLowerBound, x[:Prime,:Scottish] >= 0.6*sum(x[i,:Scottish] for i in raw))
#Scottish Club must have at most 20% standard raw whiskey

@constraint(question2, SSUpperBound, x[:Standard, :Scottish] <= 0.2*sum(x[i,:Scottish] for i in raw))
#Johnny Gold must have at most 60% standard raw whiskey

@constraint(question2, JSUpperBound, x[:Standard, :Johnny] <= 0.6*sum(x[i,:Johnny] for i in raw))
#Johnny Gold must have at Least 15% standard prime whiskey

@constraint(question2, JPUpperBound, x[:Prime,:Johnny] >= 0.15*sum(x[i,:Johnny] for i in raw))

#Sales minust cost
@objective(question2, Max, sum(sellAt[j]*sum(x[i,j] for i in raw) for j in blended) - sum(purchaseAt[i]*sum(x[i,j] for j in blended) for i in raw)- sum(y[j] for j in blended))

print(question2)

In [23]:
optimize!(question2)
print(value.(x)) # amount of blended whiskey
print(value.(y)) # advertising money
print(objective_value(question2)) # max profit

Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  1.6133333333e+03
HiGHS run time      :          0.01
2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:Standard, :Choice, :Prime]
    Dimension 2, [:Scottish, :Johnny]
And data, a 3×2 Matrix{Float64}:
 199.99999999999997  1000.0
 200.00000000000006   416.66666666666674
 600.0                250.01-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:Scottish, :Johnny]
And data, a 2-element Vector{Float64}:
   0.0
 853.33333333333341613.3333333333335